In [ ]:
pip install tensorflow matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

# Define paths
zip_path = '/content/drive/MyDrive/Colab Notebooks/gesture_dataset.zip'  # change if needed
extract_path = '/content/drive/MyDrive/gesture_dataset'

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


KeyboardInterrupt: 

In [ ]:
# STEP 3: Prepare Data Generators
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height = 128
img_width = 128
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    extract_path + '/thesis/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    extract_path + '/thesis/val',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    extract_path + '/thesis/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

class_names = list(train_data.class_indices.keys())


Found 13971 images belonging to 4 classes.
Found 4205 images belonging to 4 classes.
Found 2008 images belonging to 4 classes.


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 classes
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20  # Increase if accuracy isn't sufficient
)


Epoch 1/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 4380s 10s/step - accuracy: 0.9259 - loss: 0.1934 - val_accuracy: 0.8188 - val_loss: 0.7008
Epoch 2/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 519s 1s/step - accuracy: 0.9773 - loss: 0.0628 - val_accuracy: 0.8159 - val_loss: 0.6424
Epoch 3/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 495s 1s/step - accuracy: 0.9865 - loss: 0.0332 - val_accuracy: 0.8150 - val_loss: 1.2612
Epoch 4/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 494s 1s/step - accuracy: 0.9898 - loss: 0.0300 - val_accuracy: 0.7881 - val_loss: 1.2937
Epoch 5/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 491s 1s/step - accuracy: 0.9893 - loss: 0.0289 - val_accuracy: 0.8713 - val_loss: 1.3690
Epoch 6/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 520s 1s/step - accuracy: 0.9935 - loss: 0.0175 - val_accuracy: 0.8590 - val_loss: 1.3772
Epoch 7/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 489s 1s/step - accuracy: 0.9936 - loss: 0.0187 - val_accuracy: 0.8314 - val_loss: 1.4839
Epoch 8/20
437/437 ━━━━━━━━━━━━━━━━━━━━ 497s 1s/step - accuracy: 0.9951 - loss: 0.0163 - val_ac

In [ ]:
# STEP 6: Save the Model
model.save('/content/gesture_model.h5')
